In [2]:
import os
import openke
import traceback


from openke.config import Trainer, Tester
from openke.module.model import TransE
from openke.module.loss import MarginLoss
from openke.module.strategy import NegativeSampling
from openke.data import TrainDataLoader, TestDataLoader

os.environ['CUDA_VISIBLE_DEVICES'] = "0"

# DIM=50

In [3]:
INPUT_PATH = './data/3_openKE/synthesize/'
OUTPUT_EMBEDDING_PATH = './data/4_embedding/synthesize'
OUTPUT_MODEL_PATH = './data/4_embedding/synthesize/model'
MODEL_NAME = 'transE'
DIM = 150 #可改：50, 100, 150
EMBEDDING_NAME = f'{MODEL_NAME}_{DIM}.vec.json'

In [ ]:
# dataloader for training
print(f'Preparing train_dataloader...')
train_dataloader = TrainDataLoader(
	in_path = INPUT_PATH, 
	nbatches = 500,
	threads = os.cpu_count(), 
	sampling_mode = "normal", 
	bern_flag = 1, 
	filter_flag = 1, 
	neg_ent = 25,
	neg_rel = 0)

In [4]:
# define the model
print('Defining the model...')
transe = TransE(
	ent_tot = train_dataloader.get_ent_tot(),
	rel_tot = train_dataloader.get_rel_tot(),
	dim = DIM, 
	p_norm = 1, 
	norm_flag = True)

model_e = NegativeSampling(
	model = transe, 
	loss = MarginLoss(margin = 5.0),
	batch_size = train_dataloader.get_batch_size())

# train transe
trainer = Trainer(model = model_e, data_loader = train_dataloader, train_times = 500, alpha = 1.0, use_gpu = True)
trainer.run()
transe.save_checkpoint(f'{OUTPUT_MODEL_PATH}/{MODEL_NAME}_{DIM}.ckpt')
print('Finish training the model')

Defining the model...
Finish initializing...


Epoch 499 | loss: 0.826426: 100%|██████████| 500/500 [5:20:35<00:00, 38.47s/it]  


Finish training the model


In [5]:
# save the embeddings
print('Saving embedding...')
transe.save_parameters(f"{OUTPUT_EMBEDDING_PATH}/{EMBEDDING_NAME}")

Saving embedding...


### Loss and Time
##### dim = 50
Epoch 499 | loss: 11.588346: 100%|██████████| 500/500 [3:10:06<00:00, 22.81s/it]  
##### dim=150
Epoch 499 | loss: 0.826426: 100%|██████████| 500/500 [5:20:35<00:00, 38.47s/it]  
